### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### GOOGLE SHEETS

In [3]:
comp = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Competencias')

Descarga Correcta!


In [4]:
comp_type = {}
for i in comp['Country'].unique():
    comp_type[i] = comp[comp['Country'] == i]['Tipo'].unique()[0]

### QUERIES

In [5]:
q = '''SELECT cn.country_name AS Country,
       c.city_name AS City,
       a.area_name AS Area,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       tca.talon_campaign_name AS Campaign,
       COUNT(DISTINCT o.order_id) AS Confirmed,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq,
       SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount,
       SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount_Shopper
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
WHERE o.registered_date >= DATE_TRUNC(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH),MONTH)
      AND cn.country_name IN ('Argentina','Chile')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2,3,4,5'''

In [6]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 144473/144473 [00:23<00:00, 6024.33rows/s]


In [47]:
# Copio las bases
crudo = bq_crudo.copy()

### TRABAJO

In [48]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','City','Area','Month']
cols_float = [i for i in crudo.columns if i not in cols_str]
crudo[cols_float] = crudo[cols_float].astype(float)
crudo[cols_str] = crudo[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
crudo['Campaign'] = crudo['Campaign'].str.replace(' ', '')

In [49]:
# Creo las columnas para los segmentos
crudo['Benefit'] = crudo['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
crudo['Type'] = crudo['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
crudo['Segment'] = crudo['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
crudo['Level'] = crudo['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
crudo['Campaña'] = crudo['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
crudo['Automated'] = crudo['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
crudo['Budget'] = crudo['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crudo['Trial'] = crudo['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
crudo = func_iva(crudo)
# Marco las campañas a filtrar
crudo['Filtrar'] = crudo['Campaign'].apply(filtrar_cam)
crudo = crudo[crudo['Filtrar'] == 'No'].copy()

In [38]:
# Saco el IVA
used = 'Amount'
used_shopper = 'Amount_Shopper'
crudo['Spend'] = crudo[used_shopper] / (1 + crudo['IVA']) * crudo['Extra'] + crudo[used]

In [39]:
# Coloco el tipo de Competencia
crudo['Competitive Type'] = crudo['Country'].apply(lambda x: comp_type[x])
crudo['Union'] = crudo.apply(lambda x: str(x['City'])+'-'+str(x['Area']) if x['Competitive Type'] == 'CITY-AREA' else '-'+str(x['Area']),axis=1)
# Hago un merge con la competencia
crudo = crudo.merge(comp[['Union','Competencia']],on=['Union'],how='left')
crudo.replace([np.nan,np.inf,-np.inf],'NONE',inplace=True)

In [12]:
# Hago la PT Final
index = ['Month','Country','City','Area','Type','Budget','Trial','Segment','Competencia']
values = ['Confirmed','Acq','Spend']
pt = crudo.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()
pt.sort_values(['Country','City','Area','Month'],inplace=True)

In [13]:
# Filtro los Budget NONE
pt = pt[(pt['Budget'] != 'NONE')&(pt['Country'] == 'ARGENTINA')].copy()

### CARGA

In [14]:
carga(pt,'17WbrZFLD-JxfZu5d7eKuXDMxp6fbVlJ7jZgEE8h5jkc','Crudo')

Carga Correcta!
